# Compute

We oftentimes want to interact with other libraries and have them handle pandas objects.

In [ ]:
from IPython.display import Image
Image(filename='data/pydata-ecosystem.png',height=1024,width=1024) 

We are going to look at some interactions with:

- ``scikit-learn``
- ``statsmodels``
- ``numba`` & ``cython``
- ``dask``

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 8
pd.options.display.max_columns = 8

# Scikit-Learn

http://scikit-learn.org/stable/documentation.html

Scikit-Learn's algorithms all deal with numpy arrays. typically:

- data munging in pandas
- pass numpy array to an Estimator
- wrap result in a DataFrame or Series

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from sklearn.datasets import california_housing
data = california_housing.fetch_california_housing()

In [ ]:
X = pd.DataFrame(data.data, columns=data.feature_names)
X

In [ ]:
y = pd.Series(data.target)
y

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

In [ ]:
%%time
param_grid = dict(
    max_features=np.arange(2, 8),
    max_depth=[2, 4],
    min_samples_split=[5, 10, 15, 20],
)
rfc = RandomForestRegressor(n_estimators=10)
gs = GridSearchCV(rfc, param_grid, cv=5, n_jobs=-1)
gs.fit(X.values, y.values)

In [ ]:
scores = gs.grid_scores_
scores[:10]

In [ ]:
def unpack_grid_scores(scores):
    rows = []
    params = sorted(scores[0].parameters)
    for row in scores:
        mean = row.mean_validation_score
        std = row.cv_validation_scores.std()
        rows.append([mean, std] + [row.parameters[k] for k in params])
    return pd.DataFrame(rows, columns=['mean_', 'std_'] + params)

In [ ]:
scores = unpack_grid_scores(gs.grid_scores_)
scores

In [ ]:
(scores
       .pipe((sns.factorplot,'data'), x='max_features', y='mean_', hue='max_depth', col='min_samples_split')
 )

In [ ]:
s = pd.Series(gs.best_estimator_.feature_importances_,index=X.columns)
(s.sort_values()
  .plot
  .barh(figsize=(5,8))
)

# Statsmodels

http://statsmodels.sourceforge.net/

In [ ]:
import statsmodels
import statsmodels.api as sm
statsmodels.__version__

In [ ]:
# created in 4. Tidy Data
df = pd.read_hdf('data/games.hdf','df')
df

In [ ]:
df.info()

In [ ]:
df['home_win'] = df.home_win.astype(int)

In [ ]:
f ='home_win ~ home_strength + away_strength + home_rest + away_rest'
res = (sm
         .Logit
         .from_formula(f, df)
         .fit()
)

In [ ]:
res.summary()

In [ ]:
df2 = df.assign(rest_difference=df.home_rest - df.away_rest,
                spread=df.home_points - df.away_points)

f = 'spread ~ home_strength + away_strength + rest_difference'
res = (sm
         .OLS
         .from_formula(f, df2)
         .fit()
       )

In [ ]:
res.summary()

# Numba & Cython

http://pandas.pydata.org/pandas-docs/stable/enhancingperf.html

In [ ]:
from numba import jit
import cython
%load_ext cython

np.random.seed(1234)
pd.set_option('max_row',12)
s = Series(np.random.randn(1e5))
com = 20.0

In [ ]:
def python(s):
    output = Series(index=range(len(s)))

    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = s[0]
    output[0] = weighted_avg
    
    for i in range(1,len(s)):
        v = s[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
        
    return output

In [ ]:
%%cython
cimport cython
@cython.wraparound(False)
@cython.boundscheck(False)
def _cython(double[:] arr, double com, double[:] output):
    cdef:
        double alpha, old_weight, new_weight, weighted_avg, v
        int i
    
    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = arr[0]
    output[0] = weighted_avg
    
    for i in range(1,arr.shape[0]):
        v = arr[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
        
    return output

In [ ]:
def cython1(s):
    output = np.empty(len(s),dtype='float64')
    _cython(s.values, com, output)
    return Series(output)

In [ ]:
def cython2(s):
    return pd.ewma(s,com=com,adjust=True)

In [ ]:
@jit
def _numba(arr, output):
    alpha = 1. / (1. + com)
    old_weight = 1.0
    new_weight = 1.0
    weighted_avg = arr[0]
    output[0] = weighted_avg
    
    for i in range(1,arr.shape[0]):
        v = arr[i]
        old_weight *= (1-alpha)
        weighted_avg = ((old_weight * weighted_avg) + 
                        (new_weight * v)) / (old_weight + new_weight)
        old_weight += new_weight
        output[i] = weighted_avg
    

def numba(s):
 
    output = np.empty(len(s),dtype='float64')
    _numba(s.values, output)
    return Series(output)

In [ ]:
result1 = python(s)
result2 = cython1(s)
result3 = cython2(s)
result4 = numba(s)
result1.equals(
    result2) and result1.equals(
    result3) and result1.equals(
    result4)

In [ ]:
%timeit python(s)

In [ ]:
%timeit cython1(s)

In [ ]:
%timeit cython2(s)

In [ ]:
%timeit numba(s)

# Dask

https://dask.readthedocs.org/en/latest/

In [ ]:
import dask.dataframe as dd
from dask import threaded, multiprocessing

In [ ]:
np.random.seed(1234)
N = int(1e7)
df = DataFrame({'key' : np.random.randint(0,1000,size=N), 
                'value' : np.random.randn(N)})
ddf = dd.from_pandas(df, npartitions=8)
ddf

In [ ]:
%timeit df.groupby('key').value.sum()

In [ ]:
%timeit ddf.groupby('key').value.sum().compute(get=threaded.get)